# Preprocessing clinical data

We do this prior to upload because of patient merging issues.

In [1]:
import pandas as pd
import numpy as np
from redcap_preprocessing import redcap_preprocessing

redcap_filepath = '/Users/gustaveronteix/Documents/data/OraklOnco/clinical/20240403/raw/ColonGR-PDO_DATA_2024-03-29_1424.csv'
output_dir = '/Users/gustaveronteix/Documents/data/OraklOnco/clinical/20240403/single_files'
conversion_table_filepath = 'conversion_table/redcap_CRC_conversion_table.csv'
disease_type = 'CRC'

redcap_preprocessing.preprocess_redcap_data(redcap_filepath,
                                            disease_type=disease_type,
                                            output_dir=output_dir,
                                            save_as_single_file = False)

Detected encoding: UTF-8-SIG
Index(['record_id', 'redcap_repeat_instrument', 'redcap_repeat_instance',
       'cohorte___1', 'cohorte___2', 'cohorte___3', 'cohorte___4',
       'cohorte___5', 'sexe', 'dob',
       ...
       'hit_chimiogramme_drug___23', 'hit_chimiogramme_drug___24',
       'hit_chimiogramme_drug___25', 'hit_chimiogramme_drug___26',
       'hit_chimiogramme_drug___27', 'hit_chimiogramme_drug___28',
       'excel_chimiogramme', 'ctb_yes_no', 'date_ctb', 'organoides_complete'],
      dtype='object', length=493)
Formatting dates in column date_birth
26    1989-09-29
Name: date_birth, dtype: object
Formatting dates in column date_diagnosis
26    2020-06-30
Name: date_diagnosis, dtype: object
Formatting dates in column date_death
26    2022-05-18
Name: date_death, dtype: object
Formatting dates in column date_cell_line
26    2022-03-25
Name: date_cell_line, dtype: object
Formatting dates in column date_primitive_surgery
26    2020-11-04
Name: date_primitive_surgery, dtype: 

In [2]:
from redcap_preprocessing import redcap_preprocessing

redcap_filepath = '/Users/gustaveronteix/Documents/data/OraklOnco/clinical/20240403/raw/PANCREASGR-PDO_DATA_2024-03-29_1423.csv'
redcap_filepath = '/Users/gustaveronteix/Documents/data/OraklOnco/clinical/20240403/raw/ALICECHECK_PANCREASGR-PDO_DATA_2024-03-29_1423.csv'

output_dir = '/Users/gustaveronteix/Documents/data/OraklOnco/clinical/20240403/single_files'
conversion_table_filepath = 'conversion_table/redcap_PGR_conversion_table.csv'
disease_type = 'PDAC'

redcap_preprocessing.preprocess_redcap_data(redcap_filepath,
                                            disease_type=disease_type,
                                            output_dir=output_dir,
                                            save_as_single_file = False)

Detected encoding: Windows-1254
Index(['record_id', 'redcap_repeat_instrument', 'redcap_repeat_instance',
       'sexe', 'dob', 'dod', 'age_diagnosis', 'poids_diag', 'taille_diag',
       'deces_ou_perte_de_vue',
       ...
       'datectb', 'ctbproposal', 'specifymutation', 'frozen_pdo', 'dnarna_pdo',
       'moleculardata_pdo___1', 'moleculardata_pdo___2',
       'moleculardata_pdo___3', 'histo_pdo', 'organodes_complete'],
      dtype='object', length=315)
There are no PDO cell lines for record_id 12.
There are no PDO cell lines for record_id 12.
There are no PDO cell lines for record_id 101.
There are no PDO cell lines for record_id 101.
There are no PDO cell lines for record_id 450.
There are no PDO cell lines for record_id 450.
There are no PDO cell lines for record_id 1174.
There are no PDO cell lines for record_id 1174.
Detected encoding: Windows-1254
There are no PDO cell lines for record_id 12.
There are no PDO cell lines for record_id 12.
There are no PDO cell lines for recor